In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Defining the model training function:
To train the model we will need a directory of patches for training and a directory of patches for validation. These should have been generated from separate images (validation patches should not come from the same image as a training patch). Within each directory there should be positive (with colony) and negative (agar/background only) directories.

In [ ]:
train_data_dir = './patches/train'
val_data_dir = './patches/val'

def train_model(model, X_train_imgs, batch_size=32, epochs=50, augmented=True, reweight_classes=False):

  model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss = 'binary_crossentropy',
    metrics=['accuracy']
  )
  print('compiled')
  datagen_args = {}
  if augmented:

    datagen_args = {
        # Could augment by flipping horizontally & changing brightness, but that reduces performance
#         'rotation_range': 10,
#         'horizontal_flip': True,
#         'brightness_range': [0.5, 1.5],
        'rescale':1. / 255,
        'featurewise_center': True
    }
  else:
    datagen_args = {'rescale':1. / 255,
        'featurewise_center': True}
    
    
  # Let's define our generators etc
  train_datagen = ImageDataGenerator(**datagen_args)
  val_datagen = ImageDataGenerator(**datagen_args)

  # Fit the generators
  train_datagen.fit(X_train_imgs)
  val_datagen.fit(X_train_imgs)
  

  train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(DEFAULT_IMG_SIZE, DEFAULT_IMG_SIZE),
    color_mode = 'grayscale',
    batch_size=batch_size,
    class_mode = 'binary')

  val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(DEFAULT_IMG_SIZE, DEFAULT_IMG_SIZE),
    color_mode = 'grayscale',
    batch_size=batch_size,
    class_mode = 'binary')
    
  fit_generator_args = {}
  if reweight_classes:
    fit_generator_args = {
        'class_weight': class_weights_dict
    }
    

  callback = tf.keras.callbacks.EarlyStopping(
      monitor='val_loss', patience=3, restore_best_weights=True)
    
  # fits the model on batches with real-time data augmentation:
  history = model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_patches / batch_size,
    epochs=epochs,
    validation_data =  val_generator,
    callbacks = [callback],
    **fit_generator_args 
  )

  return history, train_datagen

Prepare a sample of the data for standardization-load in however many patches the model mean will be fit on (for featurewise_center) and convert to tensors

In [ ]:
train_patch_sample = []
count = 0
for path in train_image_paths:
    test_patch = plt.imread(path)
    test_patch = np.array(test_patch)
    test_patch = tf.expand_dims(test_patch, 2)
    train_patch_sample.append(test_patch)
    count += 1
    if count >= 10000:
        break

train_patch_sample_2 = tf.convert_to_tensor(train_patch_sample)

Generate a 'class_weights_dict' from sklearn.utils

In [ ]:
# Define the model
cnn_model_5 = models.Sequential([
  layers.Conv2D(16, (3, 3), 
                kernel_regularizer=tf.keras.regularizers.l2(0.01), 
                bias_regularizer=tf.keras.regularizers.l2(0.01), 
                input_shape=(DEFAULT_IMG_SIZE, DEFAULT_IMG_SIZE, 1)),
  layers.LeakyReLU(),
  layers.MaxPooling2D(),
  layers.Conv2D(32, (3, 3), activation='relu',
               kernel_regularizer=tf.keras.regularizers.l2(0.01), 
                bias_regularizer=tf.keras.regularizers.l2(0.01),), 
  layers.MaxPooling2D((2, 2)),
  layers.Dropout(rate=0.2),
  layers.Flatten(),
  layers.Dropout(rate=0.2),
  layers.Dense(32, activation='relu',),
  layers.Dense(1, activation='sigmoid',)
  
])
print(cnn_model_5.summary())

Train the model:

In [ ]:
[histories_cnn_5, cnn_5_train_datagen] = train_model(cnn_model_5, train_patch_sample_2, batch_size=16, epochs=15, augmented=False, reweight_classes=True)